<a href="https://colab.research.google.com/github/Taramas73/DS-final-project/blob/irusha/Segment_Anything_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Working with Meta's SAM Model
For implementing SAM, you can use the Segment Anything Model from Meta:

In [5]:
# First install the segment-anything package
!pip install git+https://github.com/facebookresearch/segment-anything.git

import numpy as np
import torch
from segment_anything import sam_model_registry, SamPredictor

def setup_sam():
    # Download the model checkpoint
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

    # Load the SAM model
    sam_checkpoint = "sam_vit_h_4b8939.pth"
    model_type = "vit_h"
    device = "cuda" if torch.cuda.is_available() else "cpu"

    sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
    sam.to(device=device)

    return SamPredictor(sam)

def segment_image(predictor, image, points=None, boxes=None):
    """
    Use SAM to segment an image
    - points: Optional list of points to guide segmentation, shape Nx2
    - boxes: Optional list of boxes to guide segmentation, shape Nx4
    """
    predictor.set_image(image)

    if points is not None:
        # Convert points to numpy arrays if they aren't already
        input_points = np.array(points)
        input_labels = np.ones(len(points))  # 1 for foreground points
        masks, scores, logits = predictor.predict(
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=True
        )
    elif boxes is not None:
        # Convert boxes to numpy arrays if they aren't already
        input_boxes = np.array(boxes)
        masks, scores, logits = predictor.predict(
            box=input_boxes[0],  # taking the first box if multiple are provided
            multimask_output=True
        )
    else:
        # Automatic segmentation based on image features
        masks, scores, logits = predictor.predict(
            multimask_output=True
        )

    return masks, scores, logits

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-9qfka2nz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-9qfka2nz
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=e8ea1fa3b107d1878f4e80148ba9cc87fa28cf9db81d29386af14053fe169874
  Stored in directory: /tmp/pip-ephem-wheel-cache-_uyfft3p/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything




* SAM requires significant compute resources, especially for the larger models
* Consider using interactive prompts for better segmentation results

* SAM can be used both for generating training data and for inference


